# 01. LTV & CAC 분석 실습

SQL로 직접 LTV와 CAC를 계산해봅니다.

**규칙:**
- 힌트는 최대한 안 보고 먼저 시도하세요
- 막히면 힌트를 펼쳐보세요
- 완료 후 `solution.sql`과 비교하세요

## 환경 설정

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

# 데이터베이스 연결
DB_PATH = Path("../data/crm.db")
conn = sqlite3.connect(DB_PATH)

# SQL 실행 헬퍼 함수
def sql(query):
    """SQL 쿼리 실행 및 결과 반환"""
    return pd.read_sql(query, conn)

print("데이터베이스 연결 완료!")

---
## 준비: 테이블 구조 파악

분석 전에 항상 데이터를 먼저 확인하세요.

In [ ]:
# customers 테이블 확인
sql("SELECT * FROM customers LIMIT 5")

In [ ]:
# transactions 테이블 확인
sql("SELECT * FROM transactions LIMIT 5")

In [ ]:
# campaigns 테이블 확인
sql("SELECT * FROM campaigns LIMIT 5")

In [ ]:
# 데이터 규모 확인
sql("""
SELECT 
    (SELECT COUNT(*) FROM customers) as total_customers,
    (SELECT COUNT(*) FROM transactions) as total_transactions,
    (SELECT COUNT(*) FROM campaigns) as total_campaigns
""")

---
# Mission 1: 기본 지표 계산

LTV 계산에 필요한 기본 지표를 먼저 구해봅시다.

## Mission 1-1: AOV (Average Order Value) 계산

**문제:** 평균 주문 금액을 계산하세요.

```
AOV = 총 매출 / 총 주문 수
```

In [ ]:
# TODO: AOV를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    _____ as total_revenue,
    _____ as total_orders,
    _____ as aov
FROM transactions
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- 총 매출: `SUM(amount)`
- 총 주문 수: `COUNT(*)`
- AOV: `SUM(amount) / COUNT(*)`
- 또는 `AVG(amount)`를 직접 사용해도 됩니다

</details>

## Mission 1-2: 고객당 평균 구매 빈도

**문제:** 고객 1명이 평균적으로 몇 번 구매하는지 계산하세요.

```
평균 구매 빈도 = 총 주문 수 / 총 고객 수
```

In [ ]:
# TODO: 평균 구매 빈도를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    _____ as total_orders,
    _____ as total_customers,
    _____ as avg_purchase_frequency
FROM transactions t
JOIN customers c ON t.customer_id = c.customer_id
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- 고유 고객 수는 `COUNT(DISTINCT customer_id)`로 구합니다
- 나눗셈 시 정수 나눗셈 주의: `CAST(... AS FLOAT)` 또는 `* 1.0`

</details>

---
# Mission 2: LTV 계산

이제 본격적으로 LTV를 계산해봅시다.

## Mission 2-1: 고객별 총 매출 (Historical LTV)

**문제:** 고객별로 지금까지의 총 매출을 계산하세요.

출력 컬럼: `customer_id`, `total_revenue`, `order_count`, `first_order_date`, `last_order_date`

In [ ]:
# TODO: 고객별 총 매출을 계산하는 SQL을 작성하세요
sql("""
SELECT 
    customer_id,
    _____ as total_revenue,
    _____ as order_count,
    _____ as first_order_date,
    _____ as last_order_date
FROM transactions
GROUP BY _____
ORDER BY total_revenue DESC
LIMIT 10
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- 총 매출: `SUM(amount)`
- 주문 수: `COUNT(*)`
- 첫 주문일: `MIN(transaction_date)`
- 마지막 주문일: `MAX(transaction_date)`
- GROUP BY는 `customer_id`로

</details>

## Mission 2-2: 전체 평균 LTV

**문제:** 전체 고객의 평균 LTV를 계산하세요.

```
평균 LTV = 전체 매출 / 전체 고객 수
```

In [ ]:
# TODO: 평균 LTV를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    SUM(amount) as total_revenue,
    COUNT(DISTINCT customer_id) as total_customers,
    _____ as avg_ltv
FROM transactions
""")

## Mission 2-3: 채널별 LTV 비교 ⭐

**문제:** 고객 획득 채널별로 평균 LTV를 비교하세요.

**비즈니스 맥락:** 어느 채널에서 온 고객이 더 가치 있는지 파악하여 마케팅 예산 배분에 활용

출력 컬럼: `acquisition_channel`, `customer_count`, `total_revenue`, `avg_ltv`

In [ ]:
# TODO: 채널별 LTV를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    c.acquisition_channel,
    _____ as customer_count,
    _____ as total_revenue,
    _____ as avg_ltv
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
GROUP BY _____
ORDER BY avg_ltv DESC
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- `LEFT JOIN`을 사용해야 구매 이력이 없는 고객도 포함됩니다
- 매출이 NULL인 경우를 대비해 `COALESCE(SUM(amount), 0)` 사용
- avg_ltv = 총매출 / 고객수

</details>

**🤔 생각해보기:**
- 가장 LTV가 높은 채널은 어디인가요?
- 그 채널에 마케팅 예산을 더 투자해야 할까요? (CAC도 고려해야 합니다!)

---
# Mission 3: CAC 계산

## Mission 3-1: 전체 CAC

**문제:** 전체 고객의 평균 CAC를 계산하세요.

```
CAC = 총 마케팅 비용 / 총 신규 고객 수
```

In [ ]:
# TODO: 전체 CAC를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    (SELECT SUM(spend) FROM campaigns) as total_marketing_cost,
    (SELECT COUNT(*) FROM customers) as total_customers,
    _____ as cac
""")

## Mission 3-2: 채널별 CAC 비교 ⭐

**문제:** 마케팅 채널별로 CAC를 계산하세요.

**비즈니스 맥락:** 채널별 효율성을 비교하여 저효율 채널 예산 축소, 고효율 채널 예산 증대

출력 컬럼: `channel`, `total_spend`, `customer_count`, `cac`

In [ ]:
# TODO: 채널별 CAC를 계산하는 SQL을 작성하세요
# 힌트: campaigns 테이블의 spend와 customers 테이블의 acquisition_channel을 활용

sql("""
WITH channel_spend AS (
    SELECT 
        channel,
        SUM(spend) as total_spend
    FROM campaigns
    GROUP BY channel
),
channel_customers AS (
    SELECT 
        acquisition_channel as channel,
        COUNT(*) as customer_count
    FROM customers
    GROUP BY acquisition_channel
)
SELECT 
    COALESCE(s.channel, c.channel) as channel,
    COALESCE(s.total_spend, 0) as total_spend,
    COALESCE(c.customer_count, 0) as customer_count,
    _____ as cac
FROM channel_spend s
FULL OUTER JOIN channel_customers c ON s.channel = c.channel
ORDER BY cac DESC
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- SQLite는 FULL OUTER JOIN을 지원하지 않습니다
- 대신 LEFT JOIN + UNION + RIGHT JOIN 또는 다른 방법 사용
- 간단하게 하려면 customer 테이블 기준으로만 계산해도 됩니다
- organic 채널은 spend가 0이므로 CAC = 0입니다

</details>

In [ ]:
# SQLite용 대안 쿼리
sql("""
SELECT 
    c.acquisition_channel as channel,
    COUNT(*) as customer_count,
    SUM(c.acquisition_cost) as total_cost,
    AVG(c.acquisition_cost) as cac
FROM customers c
GROUP BY c.acquisition_channel
ORDER BY cac DESC
""")

---
# Mission 4: LTV:CAC 비율 분석 ⭐⭐

**문제:** 채널별로 LTV:CAC 비율을 계산하고 평가하세요.

| 비율 | 등급 |
|------|------|
| < 1 | Critical |
| 1-3 | Warning |
| 3-5 | Good |
| > 5 | Excellent |

In [ ]:
# TODO: 채널별 LTV:CAC 비율을 계산하는 SQL을 작성하세요
sql("""
WITH channel_ltv AS (
    SELECT 
        c.acquisition_channel as channel,
        COUNT(DISTINCT c.customer_id) as customer_count,
        COALESCE(SUM(t.amount), 0) as total_revenue,
        COALESCE(SUM(t.amount), 0) * 1.0 / COUNT(DISTINCT c.customer_id) as avg_ltv
    FROM customers c
    LEFT JOIN transactions t ON c.customer_id = t.customer_id
    GROUP BY c.acquisition_channel
),
channel_cac AS (
    SELECT 
        acquisition_channel as channel,
        AVG(acquisition_cost) as cac
    FROM customers
    GROUP BY acquisition_channel
)
SELECT 
    l.channel,
    l.customer_count,
    ROUND(l.avg_ltv, 2) as avg_ltv,
    ROUND(c.cac, 2) as cac,
    _____ as ltv_cac_ratio,
    CASE 
        WHEN _____ THEN 'Excellent'
        WHEN _____ THEN 'Good'
        WHEN _____ THEN 'Warning'
        ELSE 'Critical'
    END as grade
FROM channel_ltv l
JOIN channel_cac c ON l.channel = c.channel
ORDER BY ltv_cac_ratio DESC
""")

<details>
<summary>💡 힌트 (클릭해서 펼치기)</summary>

- LTV:CAC 비율 = avg_ltv / cac
- CAC가 0인 경우 (organic) 처리: `CASE WHEN cac = 0 THEN NULL ELSE avg_ltv / cac END`
- CASE 문으로 등급 분류:
  - `WHEN ratio > 5 THEN 'Excellent'`
  - `WHEN ratio > 3 THEN 'Good'`
  - `WHEN ratio > 1 THEN 'Warning'`

</details>

---
# Mission 5: 실무 시나리오 ⭐⭐⭐

**시나리오:**

> 마케팅팀에서 다음 분기 예산 배분 전략을 수립하려고 합니다.
> 현재 채널별 성과 데이터를 분석하여 다음을 도출하세요:
>
> 1. 가장 효율적인 채널 (LTV:CAC 기준)
> 2. 개선이 필요한 채널
> 3. 예산 재배분 권고안

**당신의 분석을 아래에 작성하세요:**

In [ ]:
# 자유롭게 분석 쿼리를 작성하세요

# 1. 채널별 전체 성과 요약
sql("""
-- 여기에 쿼리 작성

""")

### 나의 분석 결과 및 권고안

(여기에 분석 결과를 작성하세요)

1. **가장 효율적인 채널**: 
2. **개선이 필요한 채널**: 
3. **권고안**: 

---
# 회고

실습을 마치며 아래 질문에 답해보세요.

1. **어떤 부분이 어려웠나요?**

2. **새롭게 배운 SQL 기법은?**

3. **실무에서 이 분석을 활용한다면?**

4. **면접에서 이 내용을 어떻게 설명하겠습니까?**

In [ ]:
# 데이터베이스 연결 종료
conn.close()
print("실습 완료!")